## Phonix and Soundex Python implementation
This is an implementation of the phonix phonetic search algorith [1,2]. This follows the perl[3] and [4] C implementations.

Phonix (phonetic indexing) is a technique based on Soundex, but to which
'phonetic substitution' has been added as an integral part of both the encoding
and the retrieval processes [1]. It is fairly complex, consisting of about 100-160 rules (several rules can be collapsed if they are described using regular expressions. This causes wildly different reports on the number of rules in the litterature)

The main jist of the algorithm is that rules based on phonetic spelling are applied to the search and target strings, after which the initial character is saved and all other characters are represented by a numeric value depending on which of 8 groups it belongs to. Finally the 0-group is pruned.

Phonix is the same as soundex, only with different groups and a pre-processing step that applies the rules. If we use the soundex groups and skip the pre-processing, we have soundex. Since this can be achieved in about 3 lines of code, I have decided to include soundex here, even though it is widely available.

### Outline of phonix algorithm

This algorithm maps a string `name` -> string `phonix code`, consisting of 1 letter followed by 3 digits. The algorithm assumes all characters in `name` are Alphabetic.

1. Apply ~100 transformations to `name` in order to make spelling more phonetic (see the `rules` list in code).
2. Each letter in the alphabet is substituted to with a numerical number according to one fo 8 groups the letter belongs to (see phonix_digits in code).
3. Reinstate the first letter with its alphabetic value (Use letter from after transformations are applied)
4. If first letter is a vowel or y, change the first letter to 'v'
5. Truncate all consecutive numbers and all zeros.
6. if `phonix code` is less than 4 characters long, extend it with zeros.

**Note** The retrieval process is not implemented

---

(C) Copyright 2015, Mads Olsgaard, http://olsgaard.dk
released under [BDS 3](http://opensource.org/licenses/BSD-3-Clause)

---

1. Gadd, T. N. “‘Fisching Fore Werds’: Phonetic Retrieval of Written Text in Information Systems.” Program 22, no. 3 (1988): 222–37.
2. ———. “PHONIX: The Algorithm.” Program 24, no. 4 (1990): 363–66.
3. https://github.com/maros/Text-Phonetic/blob/master/lib/Text/Phonetic/Phonix.pm
4. soundex.c in [freeWAIS-sf-2.2.10.tar.gz](https://github.com/walkingintopeople/freeWAIS/raw/master/wais/freeWAIS-sf-2.2/freeWAIS-sf-2.2.10.tar.gz)


In [1]:
import re

In [2]:
# init variables and rules

vowel = '[AEIOU]';
consonant = '[BCDFGHJLMNPQRSTVXZXY]';



# Define the letter groups

                # ABCDEFGHIJKLMNOPQRSTUVWXYZ
phonix_digits =  '01230720022455012683070808'
soundex_digits = '01230120022455012623010202'



# list of transformation/substitution rules
#            [pattern, substitution]

rules = [    [re.compile(r'[^A-Z]'), r''], #Remove all non-alphabet characters. Note that name should be uppercased before applying rules
             
             [re.compile(r'DG'),    r'G'],
             [re.compile(r'C(?=[OAU])'),    r'K'],  # Covers several rules in soundex.c [CO, CA, CU]
             [re.compile(r'C[YI]'),    r'SI'],    # Covers [CY, CI]
             [re.compile(r'CE'),    r'SE'],
             [re.compile(r'^CL(?={})'.format(vowel) ),    r'KL'],
             [re.compile(r'CK'),    r'K'],
             [re.compile(r'[GJ]C$'),    r'K'],
             [re.compile(r'^CH?R(?={})'.format(vowel)),    r'KR'],
             [re.compile(r'^WR'),    r'R'],
             [re.compile(r'NC'),    r'NK'],
             [re.compile(r'CT'),    r'KT'],
             [re.compile(r'PH'),    r'F'],
             [re.compile(r'AA'),    r'AR'], 
             [re.compile(r'SCH'),    r'SH'],
             [re.compile(r'BTL'),    r'TL'],
             [re.compile(r'GHT'),    r'T'],
             [re.compile(r'AUGH'),    r'ARF'],
             [re.compile(r'(?<={0})LJ(?={0})'.format(vowel)),    r'LD'], #
             [re.compile(r'LOUGH'),    r'LOW'],
             [re.compile(r'^Q'),    r'KW'],
             [re.compile(r'^KN'),    r'N'],
             [re.compile(r'GN$'),    r'N'],
             [re.compile(r'GHN'),    r'N'],
             [re.compile(r'GNE$'),   r'N'],
             [re.compile(r'GHNE'),   r'NE'],
             [re.compile(r'GNES$'),  r'NS'],
             [re.compile(r'^GN'),    r'N'],
             [re.compile(r'(?<=\w)GN(?={})'.format(consonant)),    r'N'],
             [re.compile(r'^PS'),    r'S'],
             [re.compile(r'^PT'),    r'T'],
             [re.compile(r'^CZ'),    r'C'],
             [re.compile(r'(?<={})WZ(?=\w)'.format(vowel)),    r'Z'],
             [re.compile(r'(?<=\w)CZ(?=\w)'),    r'CH'],
             [re.compile(r'LZ'),    r'LSH'],
             [re.compile(r'RZ'),    r'RSH'],
             [re.compile(r'(?<=\w)Z(?={})'.format(vowel)),    r'S'],
             [re.compile(r'ZZ'),    r'TS'],
             [re.compile(r'(?<={})Z(?=\w)'.format(consonant)),    r'TS'],
             [re.compile(r'HROUGH'),    r'[REW]'],
             [re.compile(r'OUGH'),    r'OF'],
             [re.compile(r'(?<={0})Q(?={0})'.format(vowel)),    r'KW'],
             [re.compile(r'(?<={0})J(?={0})'.format(vowel)),    r'Y'],
             [re.compile(r'^YJ(?={})'.format(vowel)),    r'Y'],
             [re.compile(r'^GH'),    r'G'],
             [re.compile(r'(?<={})GH$'.format(vowel),    r'E'],
             [re.compile(r'^CY'),    r'S'],
             [re.compile(r'NX'),    r'NKS'],
             [re.compile(r'^PF'),    r'F'],
             [re.compile(r'DT$'),    r'T'],
             [re.compile(r'(?<=[TD])L$'),    r'IL'], # Combines the TL and DL rules
             [re.compile(r'YTH'),    r'ITH'],
             [re.compile(r'^TS?J(?={})'.format(vowel)),    r'CH'], #combines the TJ and TSJ rules
             [re.compile(r'^TS(?={})'.format(vowel)),    r'T'],
             [re.compile(r'TCH'),    r'CHE'],
             [re.compile(r'(?<={})WSK'.format(vowel)),    r'VSIKE'],
             [re.compile(r'^[PM]N(?={})'.format(vowel)),    r'N'],
             [re.compile(r'(?<={})STL'.format(vowel)),    r'SL'],
             [re.compile(r'TNT$'),    r'ENT'],
             [re.compile(r'EAUX$'),    r'OH'],
             [re.compile(r'EXCI'),    r'ECS'],
             [re.compile(r'X'),    r'ECS'],
             [re.compile(r'NED$'),    r'ND'],
             [re.compile(r'JR'),    r'DR'],
             [re.compile(r'EE$'),    r'EA'],
             [re.compile(r'ZS'),    r'S'],
             [re.compile(r'(?<={0})H?R(?={1})'.format(vowel, consonant)),    r'AH'], # combines R and HR rule
             [re.compile(r'(?<={})HR$'.format(vowel)),    r'AH'], 
             [re.compile(r'RE$'),    r'AR'],
             [re.compile(r'(?<={})R$'.format(vowel)),    r'AH'],
             [re.compile(r'LLE'),    r'LE'],
             [re.compile(r'(?<={})LE(S?)$'.format(consonant)),    r'ILE\1'], #combines LE and LES rules
             [re.compile(r'E$'),    r''],
             [re.compile(r'ES$'),    r'S'],
             [re.compile(r'(?<={})SS$'.format(vowel)),    r'AS'],
             [re.compile(r'(?<={})MB$'.format(vowel)),    r'M'],
             [re.compile(r'MPTS'),    r'MPS'], #Why not just change to MS, if the next rule will do it anyway?
             [re.compile(r'MPS'),    r'MS'],
             [re.compile(r'MPT'),    r'MT'],
             [re.compile(r'ES$'),    r'S'], #step d in algorithm. Does not affect earlier steps
             [re.compile(r'({})$'.format(vowel)),    r'\1E'] #Step e in algorithm. Does not affect earlier steps
        ]

In [3]:
#functions

def _encode(name, digits, len=4):
    # The encoding step of phonix is the same as the encoding step of
    # soundex, except other codes are used.
    
    # name should be uppercased before calling this function!
    
    key = ''

    # translate alpha chars in name to soundex digits
        
    ord_A = 65 #No need to call ord everytime
    
    for c in name[1:]:
        if c.isalpha():
            d = digits[ord(c)-ord_A]
            
            # duplicate consecutive soundex digits are skipped
            if not key or (d != key[-1]):
                key += d

    # Insert the first character
    key = name[0] + key

    # remove all 0s from the soundex code
    key = key.replace('0','')

    # return soundex code padded to len characters
    return (key + (len * '0'))[:len]


def soundex(name):
    return _encode(name.upper(), soundex_digits)


def phonix(name):
    name = name.upper()
    for rule in rules:
        #Apply all rules sequentially to name
        name = rule[0].sub(rule[1], name)
    code = _encode(name, phonix_digits)
    
    return name, code
    

In [4]:
# Do some sanity tests
# Names and codes taken from "Data-centric systems and applications" By Peteer Christensen, Springer 2012

test_names = ['peter', 'pete', 'pedro', 'stephen', 'steve', 'smith', 'smythe', 'gail', 'gayle', 'christine', 
              'christina', 'kristina']
soundex_codes = ['p360', 'p300', 'p360', 's315', 's310', 's530', 's530', 'g400', 'g400', 'c623', 'c623', 
                 'k623']
phonix_codes = ['p300', 'p300', 'p360', 's375', 's370', 's530', 's530', 'g400', 'g400', 'c683', 'c683', 
                'k683']

print 'Name, \t soundex_code, \ttrue, \tphonix,    phonix code,\ttrue\n'
for i, n in enumerate(test_names):
    p = phonix(n)
    print '\t'.join([n+'    ', soundex(n), soundex_codes[i], p[0]+'     ', p[1], phonix_codes[i]])

Name, 	 soundex_code, 	true, 	phonix,    phonix code,	true

peter    	P360	p360	PETEAH     	P300	p300
pete    	P300	p300	PET     	P300	p300
pedro    	P360	p360	PEDRO     	P360	p360
stephen    	S315	s315	STEFEN     	S375	s375
steve    	S310	s310	STEV     	S370	s370
smith    	S530	s530	SMITH     	S530	s530
smythe    	S530	s530	SMITH     	S530	s530
gail    	G400	g400	GAIL     	G400	g400
gayle    	G400	g400	GAYIL     	G400	g400
christine    	C623	c623	KRISTIN     	K683	c683
christina    	C623	c623	KRISTINA     	K683	c683
kristina    	K623	k623	KRISTINA     	K683	k683


In [35]:
test_names = ['Knight', 'Night', 'Nite', 'Write', 'Wright', 'Rite', 'White', 'Weight', 
              'Yaeger', 'Yoga', 'Eager', 'Auger']
soundex_codes = ['K523', 'N230', 'N300', 'W630', 'W623', 'R300', 'W300', 'W230',
                 'Y230', 'Y800', 'E230', 'A230']
phonix_codes = ['N53','N53','N53', 'R63', 'R63', 'R63', 'W3','W3',
                'v2', 'v2', 'v2', 'v2']

print 'Name, \t soundex_code, \ttrue, \tphonix,    phonix code,\ttrue\n'
for i, n in enumerate(test_names):
    p = phonix(n)
    print '\t'.join([n+'    ', soundex(n), soundex_codes[i], p[0]+'     ', p[1], phonix_codes[i]])

Name, 	 soundex_code, 	true, 	phonix,    phonix code,	true

Knight    	K523	K523	NIT     	N300	N53
Night    	N230	N230	NIT     	N300	N53
Nite    	N300	N300	NIT     	N300	N53
Write    	W630	W630	RIT     	R300	R63
Wright    	W623	W623	RIT     	R300	R63
Rite    	R300	R300	RIT     	R300	R63
White    	W300	W300	WHIT     	W300	W3
Weight    	W230	W230	WEIT     	W300	W3
Yaeger    	Y260	Y230	YAEGEAH     	Y200	v2
Yoga    	Y200	Y800	YOGA     	Y200	v2
Eager    	E260	E230	vAGEAH     	v200	v2
Auger    	A260	A230	vUGEAH     	v200	v2


In [36]:
phonix('Asger')

('vSGEAH', 'v820')